<a href="https://colab.research.google.com/github/sipocz/_earthQuake/blob/main/_EarthQuake_outlier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
from sklearn.ensemble import IsolationForest
import numpy as np



In [120]:
#EarthQuake
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import sweetviz as sw

#!pip install sweetviz
#import sweetviz as sw
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [140]:
df=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/features_a.csv")
df_clasters=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/train_labels.csv")
df_testvalues=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/test_a.csv")
#official_testvalues=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/test_values.csv")


In [141]:
numx=260601
#numx=50000
Y=df_clasters[["damage_grade"]]
X=df
scaler=MinMaxScaler()
Xt=X[:numx]
Xt=scaler.fit_transform(Xt)
Xt=Xt[:numx]
Y=Y[["damage_grade"]]
Y=Y[:numx]
X_train, X_test, y_train, y_test = train_test_split(Xt, Y, random_state=2,test_size=0.05)

heads=df.columns
print(len(heads))
print(len(df_testvalues.columns))
for i in df_testvalues.columns:
    if i not in heads:
        df_testvalues.drop(columns=[i], inplace=True)

print(len(df_testvalues.columns))
print(len(X_train))
print(X_train)

57
78
57
247570
[[0.65874904 0.56666667 0.69516468 ... 0.         1.         0.        ]
 [0.15169992 0.43333333 0.14926419 ... 0.         0.         1.        ]
 [0.87421335 0.2        0.40084093 ... 0.         0.         1.        ]
 ...
 [0.77991174 0.86666667 0.51857043 ... 0.         0.         1.        ]
 [0.38710284 0.16666667 0.18710582 ... 0.         0.         1.        ]
 [0.34250192 0.26666667 0.99088998 ... 0.         0.         1.        ]]


In [123]:
clf = IsolationForest(n_estimators=3, warm_start=True, max_features=3)
clf.fit(X_train)  # fit 10 trees  
#clf.set_params()  # add 10 more trees  
#clf.fit(X)  # fit the added trees 

IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=3, max_samples='auto', n_estimators=3, n_jobs=None,
                random_state=None, verbose=0, warm_start=True)

In [124]:
X_train_predict=clf.predict(X_train)
print(len(X_train))
print(X_train)

247570
[[0.65874904 0.56666667 0.69516468 ... 0.         1.         0.        ]
 [0.15169992 0.43333333 0.14926419 ... 0.         0.         1.        ]
 [0.87421335 0.2        0.40084093 ... 0.         0.         1.        ]
 ...
 [0.77991174 0.86666667 0.51857043 ... 0.         0.         1.        ]
 [0.38710284 0.16666667 0.18710582 ... 0.         0.         1.        ]
 [0.34250192 0.26666667 0.99088998 ... 0.         0.         1.        ]]


In [144]:
def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")
outlierStatistic(X_train_predict)

[1 1 1 ... 1 1 1]
A összes (247570 darabból 35805 darab outlier van. Az  14.5 %.)


In [147]:
# az official test vizsgálata outlierekre nézve:
outlier_official_predikt=clf.predict(df_testvalues)

In [148]:
outlierStatistic(outlier_official_predikt)

[ 1  1  1 ... -1 -1  1]
A összes (86868 darabból 26798 darab outlier van. Az  30.8 %.)


In [126]:
print(list(y_train[["damage_grade"]].index)[0])

171670


In [127]:
print(X_train[1])

[0.15169992 0.43333333 0.14926419 0.         0.02512563 0.06060606
 0.03333333 0.         1.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.11111111 0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         1.         0.         0.         1.
 0.         0.         0.         1.         0.         0.
 0.         1.         0.         0.         1.         0.
 0.         0.         1.         0.         0.         0.
 0.         0.         1.        ]


In [128]:

y_train["damage_grade"].values

array([2, 2, 2, ..., 2, 2, 3])

In [129]:


X_train_inliers=[X_train[inx] for inx,i in enumerate(X_train_predict) if i==1 ]
X_train_outliers=[X_train[index] for index,i in enumerate(X_train_predict) if i==-1 ]
tmp=list(y_train["damage_grade"].values)
y_train_inliers=[tmp[inx] for inx,i in enumerate(X_train_predict) if i==1 ]
y_train_outliers=[tmp[inx] for inx,i in enumerate(X_train_predict) if i==-1 ]



In [130]:
dX1=pd.DataFrame (X_train_inliers)
dY1=pd.DataFrame (y_train_inliers)
DtX=pd.DataFrame(X_test)

In [131]:
# KNN inlier modell

from xgboost import XGBClassifier  # 72.09
# max_depth=10 : 72.79857561664441



knn_inlier = XGBClassifier(verbosity=3,max_depth = 35)

[17:32:21] ======== Monitor: Learner ========
[17:32:21] GetGradient: 3.02159s, 100 calls @ 30215us
[17:32:21] PredictRaw: 0.220522s, 100 calls @ 2205us
[17:32:21] UpdateOneIter: 818.077s, 100 calls @ 8180771us
[17:32:21] ======== Monitor: GBTree ========
[17:32:21] BoostNewTrees: 784.672s, 100 calls @ 7846721us
[17:32:21] CommitModel: 30.1615s, 100 calls @ 301615us


In [133]:
print("Fit Start--")
knn_inliers.fit(dX1, dY1)
print("Fit End--")

Fit Start--


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[17:32:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16198 extra nodes, 0 pruned nodes, max_depth=35
[17:32:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 47452 extra nodes, 0 pruned nodes, max_depth=35
[17:32:38] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 43450 extra nodes, 0 pruned nodes, max_depth=35
[17:32:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16598 extra nodes, 0 pruned nodes, max_depth=35
[17:32:48] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 49164 extra nodes, 0 pruned nodes, max_depth=35
[17:32:54] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44846 extra nodes, 0 pruned nodes, max_depth=35
[17:32:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17218 extra nodes, 0 pruned nodes, max_depth=35
[17:33:05] INFO: /workspace/src/tree/updater_prune.cc:74: tree prunin

In [134]:
y_pred=knn_inliers.predict(DtX)
print("Prediction End--")

Prediction End--


In [149]:
# Inlier statistics
from sklearn.metrics import accuracy_score
accuracy =  accuracy_score(y_test, y_pred) * 100
print(f"Accuracy a testhalmazra nézve: {accuracy}")
y_pred2=knn.predict(dX1)
accuracy =  accuracy_score(y_pred2, y_train_inliers) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")



Accuracy a testhalmazra nézve: 71.6522139513468
Accuracy a betanított halmazra : 99.99102779023919


In [150]:
dX1_ol=pd.DataFrame (X_train_outliers)
dY1_ol=pd.DataFrame (y_train_outliers)
DtX=pd.DataFrame(X_test)

In [153]:
# készítsünk modell az outlierekre is
# KNN outlier modell

from xgboost import XGBClassifier  
knn_outlier = XGBClassifier(verbosity=3,max_depth = 35)

In [154]:
print("Fit Start--")
knn_outlier.fit(dX1_ol, dY1_ol)
print("Fit End--")

Fit Start--


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[19:07:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3164 extra nodes, 0 pruned nodes, max_depth=31
[19:07:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8710 extra nodes, 0 pruned nodes, max_depth=35
[19:07:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8116 extra nodes, 0 pruned nodes, max_depth=34
[19:07:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3194 extra nodes, 0 pruned nodes, max_depth=26
[19:07:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8954 extra nodes, 0 pruned nodes, max_depth=35
[19:07:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8284 extra nodes, 0 pruned nodes, max_depth=35
[19:07:32] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3232 extra nodes, 0 pruned nodes, max_depth=29
[19:07:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 

In [155]:
from sklearn.metrics import accuracy_score
y_pred_ol=knn_outlier.predict(dX1_ol)
accuracy =  accuracy_score(y_pred_ol, y_train_outliers) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")



Accuracy a betanított halmazra : 99.99720709398129
